In [ ]:
import pandas as pd

# 1. 데이터 로드
batting_df = pd.read_csv('kbo_batting_stats_by_season_1982-2025.csv')

# 2. 2020년~2025년 데이터 필터링
df = batting_df[(batting_df['Year'] >= 2020) & (batting_df['Year'] <= 2025)].copy()

# 3. 팀별, 연도별 타석(PA) 순위 계산 (주전 9명 식별용)
df['PA_Rank'] = df.groupby(['Team', 'Year'])['PA'].rank(ascending=False, method='first')

# 4. 주전(1~9위) 제외한 나머지 백업 선수들만 추출
backup_df = df[df['PA_Rank'] > 9].copy()

# 5. 컬럼 순서 재배치 및 정리 (요청하신 순서)
target_columns = [
    'Year', 'Team', 'Name', 'G', 'PA', 'H', 'HR', 'RBI', 'R', 'SB', 
    'BB', 'SO', 'AVG', 'OBP', 'SLG', 'OPS', 'wRC+', 'oWAR', 'dWAR', 'WAR'
]

# 원본 데이터에 존재하는 컬럼만 선택 (만약 없는 컬럼이 있다면 제외)
available_columns = [col for col in target_columns if col in backup_df.columns]
backup_individual = backup_df[available_columns].copy()

# 6. 팀별/연도별 요약 데이터 생성 (평균 및 합산)
summary = backup_df.groupby(['Team', 'Year']).agg({
    'Name': 'count',
    'PA': 'sum',
    'AVG': 'mean',
    'OBP': 'mean',
    'SLG': 'mean',
    'OPS': 'mean',
    'HR': 'sum',
    'WAR': 'sum'
}).reset_index()

# 7. 결과 저장 및 출력
# 개별 선수 리스트 (요청하신 컬럼 순서)
backup_individual.to_csv('kbo_backup_players_list.csv', index=False, encoding='utf-8-sig')
# 요약 통계
summary.to_csv('kbo_backup_summary_2020_2025.csv', index=False, encoding='utf-8-sig')

print("✅ 요청하신 컬럼 순서대로 백업 선수 리스트가 'kbo_backup_players_list.csv'로 저장되었습니다.")